In [28]:
import pandas as pd
from sklearn import neighbors
from sklearn.model_selection import cross_val_score

In [7]:
nomes_colunas  = ['P0','P1','P2','P3','P4','P5','P6','P7','P8','Results']
nomes_colunas

['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'Results']

In [8]:
#Importação do dataset de treino
file_path = r'Treino.csv'
df_treino = pd.read_csv(file_path,sep=';',names=nomes_colunas)
print(df_treino.shape)
display(df_treino.head())

(1264, 10)


,P0,P1,P2,P3,P4,P5,P6,P7,P8,Results
0,Results,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,2.0
2,1,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,2.0
3,1,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,2.0
4,1,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,2.0


### **Os datasets precisam estar neste padrão ao serem carregados em qualquer algoritmo**

In [10]:
# Os datasets precisam estar neste padrão ao serem carregados em qualquer algoritmo
df_treino = df_treino.iloc[1:]
df_treino = df_treino.astype(int)
df_treino

,P0,P1,P2,P3,P4,P5,P6,P7,P8,Results
1,1,1,-1,-1,1,1,1,-1,-1,2
2,1,1,-1,-1,-1,1,1,1,-1,2
3,1,1,-1,-1,-1,1,1,-1,1,2
4,1,1,-1,-1,-1,1,1,-1,-1,2
5,1,-1,1,1,1,-1,-1,1,-1,2
...,...,...,...,...,...,...,...,...,...,...
1259,-1,-1,1,-1,0,1,0,1,1,1
1260,0,1,-1,0,1,1,-1,1,-1,1
1261,1,-1,-1,-1,-1,1,1,1,1,1
1262,0,0,-1,1,1,1,1,-1,-1,1


In [3]:
# Importação dos dataset de validacao e teste
file_path = r'Validacao.csv'
df_validacao = pd.read_csv(file_path,sep=';')
display(df_validacao.head())

file_path = r'Teste.csv'
df_teste = pd.read_csv(file_path,sep=';')
display(df_teste.head())

Results
-1  1 -1  1  1 -1 -1 -1  1        2
 1  1 -1 -1 -1  1  1 -1 -1        2
   -1  1  1 -1 -1 -1  1  1        2
-1  1  1  1 -1 -1 -1  1  1        2
 1 -1  1 -1 -1  1 -1  1 -1        2

Results
-1  1 -1 -1  1  1  1 -1  1        2
 1 -1  1 -1  1  1 -1  1 -1        2
-1  1 -1  1  1 -1  1 -1  1        2
         -1 -1  1  1 -1  1        2
   -1  1  1  1 -1 -1 -1  1        2

In [4]:
df_treino.columns

Index(['Results'], dtype='object')

### **Ajustes Datasets**

In [ ]:
# # Ajuste dataset de treino
# df_treino['Results'] = df_treino['Results'].apply(lambda x: x.replace(";",",")) # Troca de ponto e vírgulas por vírgulas
# df_treino['Label'] = df_treino['Results'].apply(lambda x: x[-1]) # Criação da coluna com a classificação da jogada
# df_treino['Results'] = df_treino['Results'].apply(lambda x: x[:-2]) # Eliminação dos 2 úlitmos valores da coluna results
# df_treino.head()

In [ ]:
# # Ajuste dataset de validacao
# df_validacao['Results'] = df_validacao['Results'].apply(lambda x: x.replace(";",",")) # Troca de ponto e vírgulas por vírgulas
# df_validacao['Label'] = df_validacao['Results'].apply(lambda x: x[-1]) # Criação da coluna com a classificação da jogada
# df_validacao['Results'] = df_validacao['Results'].apply(lambda x: x[:-2]) # Eliminação dos 2 úlitmos valores da coluna results
# df_validacao.head()

In [ ]:
# # Ajuste dataset de teste
# df_teste['Results'] = df_teste['Results'].apply(lambda x: x.replace(";",",")) # Troca de ponto e vírgulas por vírgulas
# df_teste['Label'] = df_teste['Results'].apply(lambda x: x[-1]) # Criação da coluna com a classificação da jogada
# df_teste['Results'] = df_teste['Results'].apply(lambda x: x[:-2]) # Eliminação dos 2 úlitmos valores da coluna results
# df_teste.head()

In [19]:
X = df_treino.drop(columns=['Results'])
X

,P0,P1,P2,P3,P4,P5,P6,P7,P8
1,1,1,-1,-1,1,1,1,-1,-1
2,1,1,-1,-1,-1,1,1,1,-1
3,1,1,-1,-1,-1,1,1,-1,1
4,1,1,-1,-1,-1,1,1,-1,-1
5,1,-1,1,1,1,-1,-1,1,-1
...,...,...,...,...,...,...,...,...,...
1259,-1,-1,1,-1,0,1,0,1,1
1260,0,1,-1,0,1,1,-1,1,-1
1261,1,-1,-1,-1,-1,1,1,1,1
1262,0,0,-1,1,1,1,1,-1,-1


In [17]:
Y = df_treino.drop(columns=['P0','P1','P2','P3','P4','P5','P6','P7','P8']).values
Y

array([[2],
       [2],
       [2],
       ...,
       [1],
       [1],
       [1]])

In [29]:
# Definição da quantidade de vizinhos
# k = 15

# Criação do classificador
for k in range(1,21):
    clf = neighbors.KNeighborsClassifier(n_neighbors=k)

    # Treino do modelo
    clf.fit(X.values, Y)

    score = cross_val_score(clf, X.values, Y, cv=5, scoring=None)
    print("K={}, Acurácia média: {}".format(k,score.mean()))

c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

K=1, Acurácia média: 0.5993600602296254
K=2, Acurácia média: 0.5875054896794027
K=3, Acurácia média: 0.6801587301587302


c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

K=4, Acurácia média: 0.7142041533345881
K=5, Acurácia média: 0.7648660518225735
K=6, Acurácia média: 0.8060104147060668


c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

...

c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

K=16, Acurácia média: 0.8004329004329005
K=17, Acurácia média: 0.787772131250392
K=18, Acurácia média: 0.7735271974402409


c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

K=19, Acurácia média: 0.769558943471987
K=20, Acurácia média: 0.7655938264633917


c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

In [11]:
# # Teste para escolher o melhor k
# from sklearn.model_selection import cross_val_score

# k = 1
# while k <=20:
#     # Criação do classificador
#     clf = neighbors.KNeighborsClassifier(n_neighbors=k)

#     # Treino do modelo
#     clf.fit(X.values, y)

#     # Cálculo da acurácia
#     score = cross_val_score(clf, X.values, y, cv=5, scoring=None)
#     print("K={}, Acurácia média: {}".format(k,score.mean()))
#     k+= 1

In [27]:


# Cálculo da acurácia
score = cross_val_score(clf, X.values, Y, cv=5, scoring=None)
print("K={}, Acurácia média: {}".format(k,score.mean()))

c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

K=1, Acurácia média: 0.8139092791266703
K=2, Acurácia média: 0.8139092791266703


c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

K=3, Acurácia média: 0.8139092791266703
K=4, Acurácia média: 0.8139092791266703


c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

...

c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

K=16, Acurácia média: 0.8139092791266703
K=17, Acurácia média: 0.8139092791266703
K=18, Acurácia média: 0.8139092791266703


c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

K=19, Acurácia média: 0.8139092791266703
K=20, Acurácia média: 0.8139092791266703


c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\leona\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [13]:
# Obtenção dos valores de validação
X_valid = df_validacao.drop(columns=['IMC', 'Classificacao'])
X_valid = X_valid.values

In [ ]:
# Criação das predições a partir da base de validação
predicao = clf.predict(X_valid)

# Loop para exibição dos valores analisados e sua devida classificação
for i in range(0,len(X_valid)):
  print(X_valid[i], "predicao: ", predicao[i])

In [15]:
# Obtenção dos dados de teste
X_test = df_teste.drop(columns=['IMC', 'Classificacao'])
X_test = X_test.values

In [ ]:
# Plotagens apenas para comparações
display(df_teste.head())
display(df_teste.tail())

In [ ]:
# Criação das predições a partir da base de teste
predicao = clf.predict(X_test)

for i in range(0,len(X_test)):
  print(X_test[i], "predicao: ", predicao[i])